<a href="https://colab.research.google.com/github/dimaszuda/LFP_Battery_SoC_Prediction/blob/main/Hyperparameter_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from matplotlib import pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow import keras
from tensorflow.keras import layers
!pip install keras_tuner
from keras_tuner import GridSearch
from keras_tuner import HyperParameters
from tensorflow.keras.optimizers import SGD, Adam, RMSprop, Adamax
from io import StringIO

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 11.4 MB/s eta 0:00:00


In [ ]:
df = pd.ExcelFile('Data Bersih.xlsx').parse()

In [ ]:
dt = pd.ExcelFile('data test.xlsx').parse()

In [ ]:
df.head()

,CycleID,Step_Type,C_Rate,Voltage(V),Current(mA),Cap(mAh),SoC (%)
0,1,CC_Chg,0.503168,3.32923,899.479,0.00000,0.000
1,1,CC_Chg,0.503113,3.33115,899.380,0.24983,0.014
2,1,CC_Chg,0.503113,3.33414,899.380,0.49966,0.028
3,1,CC_Chg,0.503113,3.33697,899.380,0.74951,0.042
4,1,CC_Chg,0.503168,3.33905,899.479,0.99937,0.056


In [ ]:
columns = ['CycleID',
           'Current(mA)',
           'Voltage(V)',
           'Cap(mAh)']

In [ ]:
X_train = df[columns]
y_train = df['SoC (%)'].values.reshape(-1,1)

In [ ]:
scaler = MinMaxScaler()

In [ ]:
X_train = scaler.fit_transform(X_train)
X_train = pd.DataFrame(X_train, columns=columns)

In [ ]:
y_train = scaler.fit_transform(y_train)
y_train = pd.DataFrame(y_train, columns=['SoC (%)'])

In [ ]:
X_test = dt[columns]
y_test = dt['SoC (%)'].values.reshape(-1,1)

In [ ]:
X_test = scaler.fit_transform(X_test)
X_test = pd.DataFrame(X_test, columns=columns)

In [ ]:
y_test = scaler.fit_transform(y_test)
y_test = pd.DataFrame(y_test, columns=['SoC (%)'])

In [ ]:
def build_model(hp):
  model = tf.keras.models.Sequential()
  model.add(layers.Input(shape=(4)))

  for i in range(hp.Choice(name='num_layers', values=[3])):
    model.add(layers.Dense(units=hp.Choice(f'num_of_neurons_{i}', values=[4, 8, 16, 32]), 
          activation='relu'))
  model.add(layers.Dense(1, activation=hp.Choice('activation', values=['sigmoid'])))

  optimizers = hp.Choice('optimizer', values=['rmsprop'])
  learning_rate = hp.Choice('learning_rate', values=[1e-3])

  if optimizers == 'rmsprop':
    optimizer = RMSprop(learning_rate=learning_rate)
  elif optimizers == 'adam':
    optimizer = Adam(learning_rate=learning_rate)
  elif optimizers == 'sgd':
    optimizer = SGD(learning_rate=learning_rate)
  else:
    optimizer = Adamax(learning_rate=learning_rate) 

  model.compile(loss='mae',
                optimizer=optimizer,
                metrics=['mse'])
  
  return model

In [ ]:
tuner = GridSearch(
    build_model,
    objective='loss',
    executions_per_trial=2,
    directory='tuner_model2',
    project_name='Skripsi'
)

In [ ]:
tuner.search_space_summary()
tuner.search(
    X_train,
    y_train,
    epochs=8,
)

Trial 64 Complete [00h 03m 18s]
loss: 0.009037800133228302

Best loss So Far: 0.008670924697071314
Total elapsed time: 04h 13m 16s


In [ ]:
trials = tuner.oracle.get_best_trials(num_trials=64)

In [ ]:
trial_id = []
hyperparameters = []
loss = []

In [ ]:
import json

In [ ]:
for trial in trials:
  trial_id.append(trial.trial_id)
  hyperparameters.append(trial.hyperparameters.values)
  loss.append(json.dumps(trial.score))

In [ ]:
oke = pd.DataFrame(hyperparameters)

In [ ]:
oke.insert(0, 'trial_id', trial_id)

In [ ]:
oke['loss'] = loss

In [ ]:
simpan = oke.to_excel('Percobaan Model_Skripsi_Dimas_17.xlsx', index=False)

In [ ]:
try:
    files.download('Percobaan Model_Skripsi_Dimas_17.xlsx')
except:
    pass

In [ ]:
best_models = tuner.get_best_models(num_models=2)
best_hyperparameters = tuner.get_best_hyperparameters(1)[0]

In [ ]:
print(best_hyperparameters.values)

{'num_layers': 2, 'num_of_neurons_0': 32, 'num_of_neurons_1': 32, 'activation': 'tanh', 'optimizer': 'rmsprop', 'learning_rate': 0.001}


In [ ]:
print(best_models[0].summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 16)                80        
                                                                 
 dense_1 (Dense)             (None, 32)                544       
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 657
Trainable params: 657
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
data_train = best_models.fit(X_train, y_train, epoch=800, batch_size=1000, shuffle=False)

AttributeError: ignored

In [ ]:
def plot_history(history, title, xlabel, ylabel):
  plt.figure(figsize=(10, 6))
  plt.plot(history.history['loss'], 'r')
  plt.plot(history.history['mse'], 'b')
  plt.plot(history.history['val_loss'], 'g')
  plt.plot(history.history['val_mse'], 'y')
  plt.title(title)
  plt.xlabel(xlabel)
  plt.ylabel(ylabel)
  plt.legend(['mae', 'mse', 'val_mae', 'val_mse'], loc='upper right')
  plt.show()

In [ ]:
plot_history(data_train, 'Plot Training', 'epochs', 'mae, mse, val_mae dan val_mse')

In [ ]:
data_prediksi = model.predict(X_train)

In [ ]:
predicted_data = pd.DataFrame(data_prediksi, columns=['Predicted'])

In [ ]:
Voltage = X_train['Voltage(V)']

In [ ]:
data_Voltage = pd.DataFrame(Voltage, columns=['Voltage(V)'])

In [ ]:
predicted_SoC = scaler.inverse_transform(predicted_data)

In [ ]:
def plot_prediksi(step, voltage, judul, sb_x, sb_y):
  plt.figure(figsize=(15, 6))
  plt.scatter(step, voltage, c='#fc0303')
  plt.rcParams['lines.linewidth'] == 43
  plt.rcParams['lines.linestyle'] == '-'
  plt.rcParams['lines.markersize']**0.5
  plt.title(judul)
  plt.xlabel(sb_x)
  plt.ylabel(sb_y)
  plt.show()
  return plt

In [ ]:
plot_prediksi(predicted_SoC, data_Voltage, 'SoC', 'SoC Prediksi', 'Voltage(V)')